In [1]:
import tkinter as tk
from tkinter import messagebox
import json
import os

CONFIG_FILE = "categories.json"

class CategoryApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Category Manager")

        # 레이아웃 프레임
        self.frame = tk.Frame(self.root)
        self.frame.pack(padx=10, pady=10)

        # 카테고리를 담아둘 리스트
        self.categories = []

        # 체크박스(위젯)들을 담아둘 dict
        # { 카테고리명: (tk.Checkbutton 객체, tk.BooleanVar 객체), ... }
        self.checkbuttons = {}

        # JSON 파일에서 카테고리 목록 불러오기
        self.load_categories()

        # 메인 체크박스 표시 영역
        self.checkbox_frame = tk.Frame(self.frame)
        self.checkbox_frame.pack()

        # 불러온 카테고리를 체크박스로 생성
        self.create_checkboxes()

        # 새 카테고리 추가 영역
        self.new_cat_label = tk.Label(self.frame, text="새 카테고리:")
        self.new_cat_label.pack(pady=(10,0))

        self.new_cat_entry = tk.Entry(self.frame)
        self.new_cat_entry.pack()

        self.add_button = tk.Button(self.frame, text="추가", command=self.add_category)
        self.add_button.pack(pady=(5, 5))

        # 삭제 버튼
        self.delete_button = tk.Button(self.frame, text="선택된 카테고리 삭제", command=self.delete_selected)
        self.delete_button.pack(pady=(5, 5))

    def load_categories(self):
        """JSON 파일에서 카테고리 목록 불러오기"""
        if os.path.exists(CONFIG_FILE):
            with open(CONFIG_FILE, 'r', encoding='utf-8') as f:
                self.categories = json.load(f)
        else:
            self.categories = []

    def save_categories(self):
        """현재 카테고리 목록을 JSON 파일로 저장"""
        with open(CONFIG_FILE, 'w', encoding='utf-8') as f:
            json.dump(self.categories, f, ensure_ascii=False, indent=2)

    def create_checkboxes(self):
        """
        카테고리 리스트를 순회하며 체크박스 생성
        -> json보다는 csv의 column을 이용해서 관리
        """
        for cat in self.categories:
            self.add_checkbox(cat)

    def add_checkbox(self, cat_name):
        """단일 체크박스 생성 로직"""
        var = tk.BooleanVar()
        cb = tk.Checkbutton(self.checkbox_frame, text=cat_name, variable=var)
        cb.pack(anchor='w')
        self.checkbuttons[cat_name] = (cb, var)

    def add_category(self):
        """Entry에 입력된 문자열을 카테고리에 추가"""
        new_cat = self.new_cat_entry.get().strip()
        if not new_cat:
            messagebox.showwarning("주의", "카테고리 이름을 입력하세요.")
            return

        if new_cat in self.categories:
            messagebox.showinfo("안내", "이미 존재하는 카테고리입니다.")
            return

        # 메모리(리스트)에 추가
        self.categories.append(new_cat)
        # 체크박스 동적 생성
        self.add_checkbox(new_cat)
        # 저장
        self.save_categories()
        # 엔트리 초기화
        self.new_cat_entry.delete(0, tk.END)

    def delete_selected(self):
        """체크된 카테고리를 삭제"""
        # 삭제할 카테고리를 모아서 처리
        to_delete = []
        for cat_name, (cb, var) in self.checkbuttons.items():
            if var.get():  # 체크되었다면
                to_delete.append(cat_name)

        if not to_delete:
            messagebox.showinfo("안내", "삭제할 카테고리를 선택하세요.")
            return

        # 실제 삭제
        for cat_name in to_delete:
            # 체크박스 위젯 UI에서 제거
            cb, var = self.checkbuttons[cat_name]
            cb.destroy()
            # 딕셔너리에서도 제거
            del self.checkbuttons[cat_name]
            # 카테고리 리스트에서도 제거
            self.categories.remove(cat_name)

        # JSON 파일에 반영
        self.save_categories()
        messagebox.showinfo("안내", f"카테고리 {', '.join(to_delete)} 가(이) 삭제되었습니다.")



In [2]:
root = tk.Tk()
app = CategoryApp(root)
root.mainloop()